In [1]:
from flask import Flask, render_template, jsonify, request

app = Flask(__name__)
       
# %%
import shutil
import os
import cv2
import math
import cv2
from mtcnn import MTCNN
import sys, os.path
import json
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from efficientnet.keras import EfficientNetB0
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
import numpy as np
np.set_printoptions(precision=2,suppress=True)



2.4.1
[]


In [2]:
def get_filename_only(file_path):
    file_basename = os.path.basename(file_path)
    filename_only = file_basename.split('.')[0]
    return filename_only

In [3]:
def vid_to_img(base_path,filename):
    print(filename)
    if (filename.endswith(".mp4")):
        tmp_path = os.path.join(base_path, get_filename_only(filename))
        print('Creating Directory: ' + tmp_path)
        os.makedirs(tmp_path, exist_ok=True)
        print('Converting Video to Images...')
        count = 0
        video_file = os.path.join(base_path, filename)
        cap = cv2.VideoCapture(video_file)
        frame_rate = cap.get(5) #frame rate
        while(cap.isOpened()):
            frame_id = cap.get(1) #current frame number
            ret, frame = cap.read()
            if (ret != True):
                break
            if (frame_id % math.floor(frame_rate) == 0):
                print('Original Dimensions: ', frame.shape)
                if frame.shape[1] < 300:
                    scale_ratio = 2
                elif frame.shape[1] > 1900:
                    scale_ratio = 0.33
                elif frame.shape[1] > 1000 and frame.shape[1] <= 1900 :
                    scale_ratio = 0.5
                else:
                    scale_ratio = 1
                print('Scale Ratio: ', scale_ratio)

                width = int(frame.shape[1] * scale_ratio)
                height = int(frame.shape[0] * scale_ratio)
                dim = (width, height)
                new_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
                print('Resized Dimensions: ', new_frame.shape)

                new_filename = '{}-{:03d}.png'.format(os.path.join(tmp_path, get_filename_only(filename)), count)
                count = count + 1
                cv2.imwrite(new_filename, new_frame)
                #if(count == 2):
                #    break
        cap.release()
        print("Done!")

In [4]:
def crop_faces(base_path,filename):
    tmp_path = os.path.join(base_path, get_filename_only(filename))
    print('Processing Directory: ' + tmp_path)
    frame_images = [x for x in os.listdir(tmp_path) if os.path.isfile(os.path.join(tmp_path, x))]
    faces_path = os.path.join(tmp_path, 'faces')
    print('Creating Directory: ' + faces_path)
    os.makedirs(faces_path, exist_ok=True)
    print('Cropping Faces from Images...')

    for frame in frame_images:
        print('Processing ', frame)
        detector = MTCNN()
        image = cv2.cvtColor(cv2.imread(os.path.join(tmp_path, frame)), cv2.COLOR_BGR2RGB)
        results = detector.detect_faces(image)
        print('Face Detected: ', len(results))
        count = 0
        for result in results:
            bounding_box = result['box']
            print(bounding_box)
            confidence = result['confidence']
            print(confidence)
            if len(results) < 2 or confidence > 0.95:
                margin_x = bounding_box[2] * 0.3  # 30% as the margin
                margin_y = bounding_box[3] * 0.3  # 30% as the margin
                x1 = int(bounding_box[0] - margin_x)
                if x1 < 0:
                    x1 = 0
                x2 = int(bounding_box[0] + bounding_box[2] + margin_x)
                if x2 > image.shape[1]:
                    x2 = image.shape[1]
                y1 = int(bounding_box[1] - margin_y)
                if y1 < 0:
                    y1 = 0
                y2 = int(bounding_box[1] + bounding_box[3] + margin_y)
                if y2 > image.shape[0]:
                    y2 = image.shape[0]
                print(x1, y1, x2, y2)
                crop_image = image[y1:y2, x1:x2]
                new_filename = '{}-{:02d}.png'.format(os.path.join(faces_path, get_filename_only(frame)), count)
                count = count + 1
                cv2.imwrite(new_filename, cv2.cvtColor(crop_image, cv2.COLOR_RGB2BGR))
            else:
                print('Skipped a face..')
    #faces_number=os.listdir(faces_path)
    #if(faces_number < 3):
    #    print("The Video Provided has poor lighting or has no faces")
    return faces_path

In [5]:
def predict(faces_path,faces):
    model = load_model('./best_model_DFDC40_res.h5')

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['val_accuracy'])
    count=0
    classes=np.empty([1])
    for face in faces:
        img = tf.keras.preprocessing.image.load_img(os.path.join(faces_path,face), target_size=(128, 128))
        img_tensor = tf.keras.preprocessing.image.img_to_array(img)                    # (height, width, channels)
        img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
        img_tensor /= 255.  

        classes =model.predict(img_tensor)
        if(classes[0][0]>0.75):
            count+=1
        if count>=2:
            return classes
            
        print(classes)
       
    return classes

In [6]:
def execute():
        np.set_printoptions(precision=8,suppress=True)
        basepath="C:/Users/"+ "Fahad/Downloads/video_selected.mp4"
        basedir=os.path.dirname(basepath)
        print(basedir)
        filename=os.path.basename(basepath)
        print(filename)
        vid_to_img(basedir,filename)
        faces_path = crop_faces(basedir,filename)
        faces=os.listdir(faces_path)
        print(faces)
        prediction=predict(faces_path,faces)
        basepath_upd="C:/Users/Fahad/Downloads/video_selected"
        shutil.rmtree(basepath_upd)
        os.remove(basepath)
        return prediction
# %%

In [7]:
import time
#from flaskthreads import AppContextThread
@app.route('/')
def index():
    img='static/file.svg'
    return render_template('index.html',img=img)

@app.route('/loading')
def loading():
    return render_template('loading.html')

@app.route('/team')
def team():
    return render_template('team.html')  

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/loading/prediction',methods=['GET','POST'])
def prediction():
     if request.method=='GET':
        predictions=np.empty(1)
        time.sleep(2)
        predictions=execute()
        value=predictions[0][0]*100#$[0][0])
        value=round(value,2)
        if(value < 1):
            value = 3.00
        return render_template('result.html',value=value)
        

if __name__ == "__main__":
    app.run(debug=False)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2021 22:20:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 22:20:01] "GET /static/js/index.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jun/2021 22:20:01] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 22:20:01] "GET /static/file.svg HTTP/1.1" 304 -
127.0.0.1 - - [21/Jun/2021 22:20:01] "GET /static/css/bg-upload.svg HTTP/1.1" 304 -
127.0.0.1 - - [21/Jun/2021 22:20:13] "GET /loading HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 22:20:13] "GET /static/css/style.css HTTP/1.1" 200 -


C:/Users/Fahad/Downloads
video_selected.mp4
video_selected.mp4
Creating Directory: C:/Users/Fahad/Downloads\video_selected
Converting Video to Images...
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions

[2021-06-21 22:20:33,787] ERROR in app: Exception on /loading/prediction [GET]
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-eee0ab6885f3>", line 25, in prediction
    predictions=execute()
  File "<ipython-input-6-4fa909501b70>", line 13, in execute

C:/Users/Fahad/Downloads
video_selected.mp4
video_selected.mp4
Creating Directory: C:/Users/Fahad/Downloads\video_selected
Converting Video to Images...
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions

[2021-06-21 22:22:14,876] ERROR in app: Exception on /loading/prediction [GET]
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-eee0ab6885f3>", line 25, in prediction
    predictions=execute()
  File "<ipython-input-6-4fa909501b70>", line 13, in execute

C:/Users/Fahad/Downloads
video_selected.mp4
video_selected.mp4
Creating Directory: C:/Users/Fahad/Downloads\video_selected
Converting Video to Images...
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions

[2021-06-21 22:29:26,093] ERROR in app: Exception on /loading/prediction [GET]
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-eee0ab6885f3>", line 25, in prediction
    predictions=execute()
  File "<ipython-input-6-4fa909501b70>", line 13, in execute

In [10]:
print(np.array([[-1]]))

[[-1]]
